In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode
import time

Conectando ao Neo4j 

In [7]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [8]:
# Query to get theses and their repositories from the Neo4j database
# This query retrieves theses that are published by the Federal University of Pernambuco (UFPE) and have not been processed yet.
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFPE'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

# Alternative query to get theses that do not have excerpts and have been processed.
query2 = """ 
match (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFPE'})
where (not (t:Thesis)-[:BFO_0000050]-(:Exerpt)) and  exists {(t{processed:true})}
return distinct(t["uri"]) as Thesis, t["repository"] as repository
"""

# Query to retrive theses with few pages and their repositories.
query3 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFPE'})
Match (t:Thesis)-[:BFO_0000050]-(e:Exerpt)
RETURN count(e) as pages, t["uri"] as Thesis, t["repository"] as repository
 order by (pages) limit 40
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query1).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
#thesis_repository = pd.DataFrame(results, columns=["pages", "Thesis", "repository"])
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository.reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:eu_vivo_na_florestaeu_tenho_os...,https://repositorio.ufpe.br/handle/123456789/4...
1,tag:stardog:api:caminhos_para_o_reencantamento...,https://repositorio.ufpe.br/handle/123456789/4...
2,tag:stardog:api:vozes_da_periferia_na_literatu...,https://repositorio.ufpe.br/handle/123456789/5...
3,tag:stardog:api:territorializacao_e_narrativas...,https://repositorio.ufpe.br/handle/123456789/4...
4,tag:stardog:api:fluxos_de_comunicacao_fulnio__...,https://repositorio.ufpe.br/handle/123456789/5...
...,...,...
527,tag:stardog:api:metodo_adaptativo_de_markov_ch...,https://repositorio.ufpe.br/handle/123456789/4950
528,tag:stardog:api:movimento_negro_ou_movimentos_...,https://repositorio.ufpe.br/handle/123456789/3...
529,tag:stardog:api:avaliacao_de_desempenho_de_uma...,https://repositorio.ufpe.br/handle/123456789/1...
530,tag:stardog:api:corais_negros_cnidaria_anthozo...,https://repositorio.ufpe.br/handle/123456789/2...


Criando as utils functions

In [9]:
# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(url, verify=True).text#, verify=False) 
    except:
        print("Sleeping for 5 seconds before retrying...")
        time.sleep(5)
        try:
            f = requests.get(url, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying...")
            time.sleep(30)
            f = requests.get(url, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    link_list = []
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):     # (doc['href'].endswith('.pdf')):# & doc['href'].startswith('/bitstream')):
            path = doc['href']
            prefix_uni = 'https://repositorio.ufpe.br'
            link = prefix_uni + path
            link_list.append(link)
    if link_list != []:
        return list(set(link_list))
    else:
        return None
    
    #return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):


    # NOTE the stream=True parameter below
    try:
        r = requests.get(pdf_link, verify=True, stream=True)
    except:
        print("Sleeping for 5 seconds before retrying...")
        time.sleep(5)
        try:
            r = requests.get(pdf_link, verify=True, stream=True)
        except:
            print("Sleeping for 30 seconds before retrying...")
            time.sleep(30)
            r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese UFPE.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    try:
        reader = PdfReader(tese_path)
    except:
        print(f"Erro ao ler o arquivo {tese_path}. ")
        excerpt = {}
        return excerpt
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        
        # Extraindo o texto da página
        try:
            if page_number == 0:
                try:
                    page_text = (reader.pages[page_number].extract_text() + '\n ' 
                                + reader.pages[page_number+1].extract_text()[:400])
                except:
                    page_text = reader.pages[page_number].extract_text()
            else:
                
                if page_number == number_of_pages -1:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text())
                    except:
                        page_text = reader.pages[page_number].extract_text()
                
                else:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text() + '\n ' 
                                    + reader.pages[page_number+1].extract_text()[:400])  
                    except:
                        page_text = reader.pages[page_number].extract_text()
        except:
            page_text = ''
        
        # Limpando page text         
        page_text = page_text.replace("'", "").replace('"', '')
        # Verifica se o último caractere é uma barra invertida
        if page_text != '':
            if page_text[-1] == '\\': 
                page_text = page_text[:-1]
        page_text = re.sub(r"[\ud800-\udfff]", " ", page_text)
        page_text = page_text.encode('utf-8').decode('utf-8')
        
        
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        

        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [ ]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        # Itereando pelos links de PDF encontrados (em alguns casos há outros materiais além da tese)
        for pdf_num in range(len(pdf_link)):
            print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link[pdf_num]}")
            download_file(pdf_link[pdf_num], thesis_uri)

            # Extraindo o texto e entidades
            tese_path = 'tese UFPE.pdf'
            excerpt = extratc_text(tese_path)

            excerpts = []
            persons = []
            gpes = []

            # Salvando as triplas em CSV
            for n in excerpt.keys():

                excerpt_uri = thesis_repository['Thesis'][i] + '_pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_text = excerpt[n]['text']
                # Verifica se o último caractere é uma barra invertida
                #if excerpt_text[-1] == '\\': 
                #    excerpt_text = excerpt_text[:-1]
                

                excerpt_page = 'pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_lang = excerpt[n]['lang']
                
                excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, excerpt_page, excerpt_lang])
                    
                #Criando as relações com as entidades
                for person in excerpt[n]['persons']:
                    persons.append([excerpt_uri, 'tag:stardog:api:' + person])
                
                for gpe in excerpt[n]['gpes']:
                    gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
            
            # Salvando os dados em CSV no diretório de importação do Neo4j
            pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFRPE.csv', index=False, header=False)
            pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFRPE.csv', index=False, header=False)
            pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFRPE.csv', index=False, header=False)

            #Deletando o arquivo PDF após o processamento
            os.remove('tese UFPE.pdf')

            print(f"Loading triplas: {thesis_uri}")

            # Importanto os dados para o Neo4j
            #Excerpts
            query = """
            LOAD CSV FROM 'file:///excerptsUFRPE.csv'
            AS row
            MATCH  (t:Thesis{uri: row[0]})
            MERGE  (e:Exerpt{
                    uri: row[1],
                    text: row[2],
                    page: row[3],
                    lang: row[4]
                    })-[r:BFO_0000050]-(t);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #Persons
            query = """
            LOAD CSV FROM 'file:///personsUFRPE.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Person{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #GPEs
            query = """
            LOAD CSV FROM 'file:///gpesUFRPE.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Place{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            # Removendo os arquivos CSV após a importação
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFRPE.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFRPE.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFRPE.csv')

            print(f"Total time: {pd.Timestamp.now() - start_time}")
            print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
            # Atualizando a hora parcial
            parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()

0 - Downloading and extracting: eu_vivo_na_florestaeu_tenho_os_meus_ensinos__as_experiencias_com_as_plantas_professoras_entre_o_padrinho_sebastiao_mota_de_melo_e_suas_companheiras_e_companheiros_no_santo_daime from https://repositorio.ufpe.br/bitstream/123456789/49584/1/TESE%20Nathalia%20Cavalcanti%20da%20Silva.pdf
Loading triplas: eu_vivo_na_florestaeu_tenho_os_meus_ensinos__as_experiencias_com_as_plantas_professoras_entre_o_padrinho_sebastiao_mota_de_melo_e_suas_companheiras_e_companheiros_no_santo_daime
Total time: 0 days 00:00:42.647699
Partial time: 0 days 00:00:42.647699
1 - Downloading and extracting: caminhos_para_o_reencantamento_da_vida__entre_demandas_e_praticas_de_cuidado_e_acolhimento_em_espacos_academicos_da_ufpe from https://repositorio.ufpe.br/bitstream/123456789/48711/1/DISSERTA%c3%87%c3%83O%20Fabiana%20dos%20Santos%20Louro.pdf
Loading triplas: caminhos_para_o_reencantamento_da_vida__entre_demandas_e_praticas_de_cuidado_e_acolhimento_em_espacos_academicos_da_ufpe
Total

unknown widths : 
[0, IndirectObject(1277, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1283, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1289, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1295, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1301, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1307, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1313, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1319, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1325, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1331, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1337, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1343, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1349, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1355, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1361, 0, 1226517909264)]
unknown widths : 
[0, IndirectObject(1367, 0, 1226517909264)]
unknown 

Loading triplas: historias_indigenas_e_mitos_restauradores__os_potiguaras_entre_santos_festas_e_ruinas
Total time: 0 days 00:29:06.305227
Partial time: 0 days 00:00:59.938017
46 - Downloading and extracting: nao_tem_certo_nao_tem_errado__estratigrafia_das_vozes_significados_e_apropriacoes_da_cultura_material_na_comunidade_da_aldeia_da_mina_grande__ti_kapinawa_pe from https://repositorio.ufpe.br/bitstream/123456789/39557/1/DISSERTA%c3%87%c3%83O%20Mariana%20Zanchetta%20Otaviano.pdf
Loading triplas: nao_tem_certo_nao_tem_errado__estratigrafia_das_vozes_significados_e_apropriacoes_da_cultura_material_na_comunidade_da_aldeia_da_mina_grande__ti_kapinawa_pe
Total time: 0 days 00:29:27.924859
Partial time: 0 days 00:00:21.619632
47 - Downloading and extracting: os_paratextos_do_livro_traduzido__uma_experiencia_com_leitores_das_fabulas_indianas_do_pancatantra from https://repositorio.ufpe.br/bitstream/123456789/35179/1/DISSERTA%c3%87%c3%83O%20Laura%20Freitas%20da%20Fonseca%20e%20Silva.pdf
Loadi